In [ ]:
from nltk.corpus import semcor
import nltk
import numpy

nltk.download("semcor")
nltk.download('wordnet')

[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
help(semcor.tagged_sents)

Help on method tagged_sents in module nltk.corpus.reader.semcor:

tagged_sents(fileids=None, tag='pos') method of nltk.corpus.reader.semcor.SemcorCorpusReader instance
    :return: the given file(s) as a list of sentences. Each sentence
        is represented as a list of tagged chunks (in tree form).
    :rtype: list(list(Tree))
    
    :param tag: `'pos'` (part of speech), `'sem'` (semantic), or `'both'`
        to indicate the kind of tags to include.  Semantic tags consist of
        WordNet lemma IDs, plus an `'NE'` node if the chunk is a named entity
        without a specific entry in WordNet.  (Named entities of type 'other'
        have no lemma.  Other chunks not in WordNet have no semantic tag.
        Punctuation tokens have `None` for their part of speech tag.)



In [ ]:
input_seq="Jacob wants to immerse himself in some books."

In [ ]:
from nltk.corpus import semcor
tagged_sents = [[str(c) for c in s] for s in semcor.tagged_sents(tag='both')]
tagged_sents_actual = semcor.tagged_sents(tag='both')

In [ ]:
print(tagged_sents[0])
tagged_sents[2][1]

['(DT The)', '(group.n.01 (NE (NNP Fulton County Grand Jury)))', '(say.v.01 (VB said))', '(friday.n.01 (NN Friday))', '(DT an)', '(investigation.n.01 (NN investigation))', '(IN of)', '(atlanta.n.01 (NN Atlanta))', "(POS 's)", '(recent.s.02 (JJ recent))', '(primary_election.n.01 (NN primary election))', '(produce.v.04 (VB produced))', '(None ``)', '(DT no)', '(evidence.n.01 (NN evidence))', "(None '')", '(IN that)', '(DT any)', '(irregularity.n.01 (NN irregularities))', '(take_place.v.01 (VB took place))', '(None .)']


'(september.n.01 (NN September))'

In [ ]:
import gensim
import gensim.downloader as api
import json
info = api.info()
print(json.dumps(info, indent=4))
print(api.load('word2vec-google-news-300', return_path=True))

{
    "corpora": {
        "semeval-2016-2017-task3-subtaskBC": {
            "num_records": -1,
            "record_format": "dict",
            "file_size": 6344358,
            "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py",
            "license": "All files released for the task are free for general research use",
            "fields": {
                "2016-train": [
                    "..."
                ],
                "2016-dev": [
                    "..."
                ],
                "2017-test": [
                    "..."
                ],
                "2016-test": [
                    "..."
                ]
            },
            "description": "SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collect

In [ ]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', binary=True)

In [ ]:
tagged_sents[2]
tagged_dataset=[]
for sent in tagged_sents:
    my_arr=[]
    for word in sent:
        if word[1]==word[1].lower():
            #my_arr.append([word.replace("[","").replace("]","").replace("\'",""),"None"])
        #elif word[0]=="(":
            synset=word.split(" ")[0][1:]
            if word.split(" ")[1][1:] == "NE":
                pos_tag="NN"
                chunk=word.split(" ")[3:]
                chunk=" ".join(chunk).strip(")(")
            else:
                pos_tag=word.split(" ")[1][1:]
                chunk=word.split(" ")[2:]
                chunk=" ".join(chunk).strip(")(")
            #chunk=(" ").join(word.split(" ")[1:]).strip()
            #if chunk[0]=="(":
            #    chunk=chunk[1:-2].strip()
            #    if chunk[:2]=="NE":
            #        chunk=chunk[2:].strip()
            #    else:
            #        print("found other tag ", chunk[:5])
            #else:
            #    chunk=chunk[:-1].strip()
            my_arr.append([chunk,synset,pos_tag])
                
        else:
            pos_tag=word.split(" ")[0][1:]
            chunk=word.split(" ")[1][:-1]
            my_arr.append([chunk,None,pos_tag])
    tagged_dataset.append(my_arr)

In [ ]:
# try:
#         word_vec = w2v_model.wv[word]
#     except:
#         word_vec = np.random.rand(300)

# synids=wn.synsets(chunk)
# for id in synids:
#                 if id.name().split(".")[1]=="n":
#                     new_synids.append(id)
#             synids=new_synids
# if len(synids)==0 and len(chunk.split(" "))>1:
#                 # find senses for each component words of chunk and whichever scores highest take it as the sense of the whole chunk
#                 pass
#             elif len(synids)==0 and len(chunk.split(" "))==1:
#                 my_arr.append([chunk,"None"])
#             elif len(chunk.split(" "))==1:

In [ ]:
#extended lesk

from nltk.corpus import wordnet as wn
import numpy as np
nltk.download('omw-1.4')

predict_dataset=[]
for sent in tagged_dataset:
    chunks_in_sent=[]
    for tuple_ in sent:
        chunks_in_sent.append(tuple_[0])
    chunks_in_sent=" ".join(chunks_in_sent)
    context=chunks_in_sent.split(" ")
    context_vec=np.array([])
    for word in context:
        if len(context_vec)==0:
            try:
                context_vec=np.expand_dims(w2v_model.wv[word],0)
            except:
                context_vec = np.expand_dims(np.random.rand(300),0)
        else:
            try:
                context_vec=np.append(context_vec,np.expand_dims(w2v_model.wv[word],axis=0),axis=0)
            except:
                context_vec=np.append(context_vec,np.expand_dims(np.random.rand(300),0),axis=0)
    context_vec=np.mean(context_vec,axis=0)


    my_arr=[]
    for tuple_ in sent:
        
        chunk,sense,pos_tag=tuple_
        
        if pos_tag!="NN":
            my_arr.append([])
        else:
            if len(chunk.split(" "))<=1 or len(wn.synsets(chunk)) >0:
                
                synids=wn.synsets(chunk)
                new_synids=[]
                for id in synids:
                    if id.name().split(".")[1]=="n":
                        new_synids.append(id)
                synids=new_synids
                scores=[]
                for id in synids:
                    words=id.definition().split(" ")
                    for id2 in id.hypernyms():
                        for le in id2.definition().split(" "):
                            words.append(le)
                    for id2 in id.hyponyms():
                        for le in id2.definition().split(" "):
                            words.append(le)
                    
                    word_vec=np.array([])
                    for word in words:
                        if len(word_vec)==0:
                            try:
                                word_vec=np.expand_dims(w2v_model.wv[word],0)
                            except:
                                word_vec = np.expand_dims(np.random.rand(300),0)
                        else:
                            try:
                                word_vec=np.append(word_vec,np.expand_dims(w2v_model.wv[word],axis=0),axis=0)
                            except:
                                word_vec=np.append(word_vec,np.expand_dims(np.random.rand(300),0),axis=0)
                    word_vec=np.mean(word_vec,axis=0)
                    scores.append(np.dot(word_vec,context_vec).item())
                if len(scores) == 0:
                    winning_id=None
                    my_arr.append([chunk,winning_id])
                else:

                    winning_id=synids[scores.index(max(scores))]
                    my_arr.append([chunk,winning_id])
            else:
                winning_scores=[]
                winning_ids=[]
                for word_chunk in chunk:
                    synids=wn.synsets(word_chunk)
                    new_synids=[]
                    for id in synids:
                        if id.name().split(".")[1]=="n":
                            new_synids.append(id)
                    synids=new_synids
                    scores=[]
                    for id in synids:
                        words=id.definition().split(" ")
                        for id2 in id.hypernyms():
                            for le in id2.definition().split(" "):
                                words.append(le)
                        for id2 in id.hyponyms():
                            for le in id2.definition().split(" "):
                                words.append(le)
                        word_vec=np.array([])
                        for word in words:
                            if len(word_vec)==0:
                                try:
                                    word_vec=np.expand_dims(w2v_model.wv[word],0)
                                except:
                                    word_vec = np.expand_dims(np.random.rand(300),0)
                            else:
                                try:
                                    word_vec=np.append(word_vec,np.expand_dims(w2v_model.wv[word],axis=0),axis=0)
                                except:
                                    word_vec=np.append(word_vec,np.expand_dims(np.random.rand(300),0),axis=0)
                        word_vec=np.mean(word_vec,axis=0)
                        scores.append(np.dot(word_vec,context_vec).item())
                    if len(scores) == 0:
                        winning_id=None
                        winning_scores.append(-999)
                        winning_ids.append(winning_id)
                    else:

                        winning_id=synids[scores.index(max(scores))]
                        winning_scores.append(max(scores))
                        winning_ids.append(winning_id)
                
                max_score=max(winning_scores)
                max_id=winning_ids[winning_scores.index(max_score)]
                my_arr.append([chunk,max_id])
    predict_dataset.append(my_arr)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `wv` (At

KeyboardInterrupt: ignored

In [ ]:
output=[]
actual=[]
for i in range(len(tagged_dataset)):
    for j in range(len(tagged_dataset[i])):
        chunk0,synset0,tag0=tagged_dataset[i][j]
        if tag0=="NN":
            if synset0!=None:
                actual.append(synset0)
                if len(predict_dataset[i][j])==0:
                    print(tagged_dataset[i][j])
                    break
                if predict_dataset[i][j][1] is None:
                    output.append("None")
                else:
                    output.append(predict_dataset[i][j][1].name())


IndexError: ignored

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
print(precision_score(actual,output,average="weighted"))
print(recall_score(actual,output,average="weighted"))
print(f1_score(actual,output,average="weighted"))

In [ ]:
from sklearn.metrics import confusion_matrix
a=confusion_matrix(actual[:-1],output,labels=b)


In [ ]:
a.shape

(7138, 7138)

In [ ]:
print(np.amax(a))
result=np.where(a==np.amax(a))
print(result)

68
(array([2136]), array([3004]))


In [ ]:
set1=set(output)
set2=set(actual)
set3=set.union(set2)
b=list(set3)

In [ ]:
b[2136]

'type.n.01'

In [ ]:
b[3004]

'character.n.05'

'a period of time containing 365 (or 366) days'

In [ ]:
wn.synsets("type")[4].definition()

'all of the tokens of the same symbol'

In [ ]:
wn.synsets("type")[1].definition()

'a person of a specified kind (usually with many eccentricities)'